# Preparations

In [1]:
!pip -q install implicit
!pip -q install rectools==0.4.2
!pip -q install lightfm
!pip -q install nmslib
!pip -q install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.3 MB/s eta 0:00:00


In [2]:
import implicit

implicit.gpu.HAS_CUDA

True

In [3]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
import os

os.environ["OPENBLAS_NUM_THREADS"] = "1"  # For implicit ALS
import warnings

warnings.filterwarnings("ignore")

In [5]:
import typing as tp
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from implicit.lmf import LogisticMatrixFactorization
from lightfm import LightFM
from rectools import Columns
from rectools.dataset import Dataset
from rectools.metrics import (
    MAP,
    NDCG,
    MeanInvUserFreq,
    Precision,
    Recall,
    Serendipity,
    calc_metrics,
)
from rectools.model_selection import TimeRangeSplitter, cross_validate
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel, PopularModel
from rectools.tools import UserToItemAnnRecommender
from tqdm import tqdm

# Loading data

In [6]:
DATA_PATH = Path("/content/drive/MyDrive/kion_train/")
users = pd.read_csv(DATA_PATH / "users.csv")
items = pd.read_csv(DATA_PATH / "items.csv")
interactions = pd.read_csv(DATA_PATH / "interactions.csv")

In [7]:
Columns.Datetime = "last_watch_dt"
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format="%Y-%m-%d")
max_date = interactions[Columns.Datetime].max()
interactions[Columns.Weight] = np.where(interactions["watched_pct"] > 10, 3, 1)

In [8]:
# разделим датасет на три части: на валидации будем подбирать гиперпараметры, на тесте финально сравнивать модели
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

train.drop(train.query("total_dur < 300").index, inplace=True)

# отфильтруем холодных пользователей
cold_users = set(test[Columns.User]) - set(train[Columns.User])
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

TEST_USERS = test[Columns.User].unique()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (3832711, 6)
test: (333026, 6)


## Preparing features

In [9]:
def get_user_features(users: pd.DataFrame, interactions: pd.DataFrame, features: tp.List[str]):
    users.fillna("Unknown", inplace=True)
    users = users.loc[users[Columns.User].isin(interactions[Columns.User])].copy()
    user_features_frames = []
    for feature in features:
        feature_frame = users.reindex(columns=[Columns.User, feature])
        feature_frame.columns = ["id", "value"]
        feature_frame["feature"] = feature
        user_features_frames.append(feature_frame)
    user_features = pd.concat(user_features_frames)
    return user_features

In [10]:
user_features = get_user_features(users, train, ["sex", "age", "income"])

In [11]:
def get_item_features(items: pd.DataFrame, interactions: pd.DataFrame):
    items = items.loc[items[Columns.Item].isin(interactions[Columns.Item])].copy()
    items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
    genre_feature = items[["item_id", "genre"]].explode("genre")
    genre_feature.columns = ["id", "value"]
    genre_feature["feature"] = "genre"
    content_feature = items.reindex(columns=[Columns.Item, "content_type"])
    content_feature.columns = ["id", "value"]
    content_feature["feature"] = "content_type"
    item_features = pd.concat((genre_feature, content_feature))
    return item_features

In [12]:
item_features = get_item_features(items, train)

## Constructing the dataset

In [13]:
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

# Hyperparam tuning

In [14]:
import optuna
from optuna.samplers import TPESampler

optuna.logging.set_verbosity(optuna.logging.INFO)

In [15]:
K_RECOS = 10
RANDOM_STATE = 42
N_EPOCHS = 1  # Lightfm

In [16]:
def ALS_objective(trial, dataset, train, test):
    test_users = test[Columns.User].unique()
    metrics = {"MAP@10": MAP(k=10)}
    factors = trial.suggest_categorical("n_factors", [8, 16, 32])
    num_threads = trial.suggest_int("num_threads", 1, 3)
    fit_features_together = trial.suggest_categorical("fit_features_together", [True, False])

    model = ImplicitALSWrapperModel(
        model=AlternatingLeastSquares(
            factors=factors,
            random_state=RANDOM_STATE,
            num_threads=num_threads,
        ),
        fit_features_together=fit_features_together,
    )

    model.fit(dataset)
    recos = model.recommend(
        users=test_users,
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )
    metric_values = calc_metrics(metrics, recos, test, train)
    return metric_values["MAP@10"]

In [17]:
sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="ALS", direction="maximize", sampler=sampler)
study.optimize(lambda trial: ALS_objective(trial, dataset, train, test), n_trials=20)

[I 2023-12-03 07:02:22,844] A new study created in memory with name: ALS
[I 2023-12-03 07:03:15,914] Trial 0 finished with value: 0.0747821510746944 and parameters: {'n_factors': 16, 'num_threads': 1, 'fit_features_together': True}. Best is trial 0 with value: 0.0747821510746944.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[I 2023-12-03 07:03:56,642] Trial 1 finished with value: 0.06377260428766873 and parameters: {'n_factors': 32, 'num_threads': 2, 'fit_features_together': False}. Best is trial 0 with value: 0.0747821510746944.


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

[I 2023-12-03 07:09:04,136] Trial 8 finished with value: 0.0693735515675699 and parameters: {'n_factors': 8, 'num_threads': 3, 'fit_features_together': False}. Best is trial 5 with value: 0.07556742809915605.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[I 2023-12-03 07:09:44,667] Trial 9 finished with value: 0.06376552759889925 and parameters: {'n_factors': 32, 'num_threads': 1, 'fit_features_together': False}. Best is trial 5 with value: 0.07556742809915605.
[I 2023-12-03 07:10:31,891] Trial 10 finished with value: 0.0748282843304979 and parameters: {'n_factors': 8, 'num_threads': 2, 'fit_features_together': True}. Best is trial 5 with value: 0.07556742809915605.
[I 2023-12-03 07:11:25,208] Trial 11 finished with value: 0.07484846953149593 and parameters: {'n_factors': 32, 'num_threads': 2, 'fit_features_together': True}. Best is trial 5 with value: 0.07556742809915605.
[I 2023-12-03 07:12:15,463] Trial 12 finished with value: 0.07504440537346238 and parameters: {'n_factors': 16, 'num_threads': 2, 'fit_features_together': True}. Best is trial 5 with value: 0.07556742809915605.
[I 2023-12-03 07:13:07,911] Trial 13 finished with value: 0.07516940053884456 and parameters: {'n_factors': 32, 'num_threads': 2, 'fit_features_together': Tru

In [18]:
def lightfm_objective(trial, dataset, train, test):
    test_users = test[Columns.User].unique()
    metrics = {"MAP@10": MAP(k=10)}
    no_components = trial.suggest_categorical("n_factors", [8, 16, 32, 64])
    loss = trial.suggest_categorical("loss", ["logistic", "bpr", "warp"])
    learning_rate = trial.suggest_float("lr", 1e-3, 1e-1, log=True)
    num_threads = trial.suggest_int("num_threads", 1, 3)
    user_alpha = trial.suggest_float("user_alpha", 0, 1)
    item_alpha = trial.suggest_float("item_alpha", 0, 1)

    model = LightFMWrapperModel(
        LightFM(
            no_components=no_components,
            loss=loss,
            random_state=RANDOM_STATE,
            learning_rate=learning_rate,
            user_alpha=user_alpha,
            item_alpha=item_alpha,
        ),
        epochs=N_EPOCHS,
        num_threads=num_threads,
    )

    model.fit(dataset)
    recos = model.recommend(
        users=test_users,
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )
    metric_values = calc_metrics(metrics, recos, test, train)
    return metric_values["MAP@10"]

In [19]:
sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="lightFM", direction="maximize", sampler=sampler)
study.optimize(lambda trial: lightfm_objective(trial, dataset, train, test), n_trials=20)

[I 2023-12-03 07:18:18,648] A new study created in memory with name: lightFM
[I 2023-12-03 07:19:30,634] Trial 0 finished with value: 0.07652084279251593 and parameters: {'n_factors': 16, 'loss': 'warp', 'lr': 0.0049104518184659674, 'num_threads': 2, 'user_alpha': 0.538816734003357, 'item_alpha': 0.4191945144032948}. Best is trial 0 with value: 0.07652084279251593.
[I 2023-12-03 07:20:14,577] Trial 1 finished with value: 0.00019192744523331218 and parameters: {'n_factors': 32, 'loss': 'logistic', 'lr': 0.0019088591198098556, 'num_threads': 1, 'user_alpha': 0.8007445686755367, 'item_alpha': 0.9682615757193975}. Best is trial 0 with value: 0.07652084279251593.
[I 2023-12-03 07:22:29,709] Trial 2 finished with value: 0.06494712932354058 and parameters: {'n_factors': 64, 'loss': 'warp', 'lr': 0.05705385668376793, 'num_threads': 1, 'user_alpha': 0.42110762500505217, 'item_alpha': 0.9578895301505019}. Best is trial 0 with value: 0.07652084279251593.
[I 2023-12-03 07:24:40,795] Trial 3 finish

# Cross-validation

## Models

Сравним лучшие модели на кросс-валидации

In [25]:
models = {
    "popular": PopularModel(),
    "ALS": ImplicitALSWrapperModel(
        model=AlternatingLeastSquares(
            factors=32,
            random_state=RANDOM_STATE,
            num_threads=2,
        ),
        fit_features_together=True,
    ),
    "LightFM": LightFMWrapperModel(
        LightFM(
            no_components=8,
            loss="warp",
            random_state=RANDOM_STATE,
            learning_rate=0.05,
            user_alpha=0.3,
            item_alpha=0.2,
        ),
        epochs=N_EPOCHS,
        num_threads=2,
    ),
}

## Metrics

In [20]:
metrics_name = {
    "precision": Precision,
    "recall": Recall,
    "MAP": MAP,
    "NDCG": NDCG,
    "novelty": MeanInvUserFreq,
    "serendipity": Serendipity,
}

metrics = {}
for metric_name, metric in metrics_name.items():
    for k in [1, 5, 10]:
        metrics[f"{metric_name}@{k}"] = metric(k=k)

In [21]:
metrics

{'precision@1': Precision(k=1),
 'precision@5': Precision(k=5),
 'precision@10': Precision(k=10),
 'recall@1': Recall(k=1),
 'recall@5': Recall(k=5),
 'recall@10': Recall(k=10),
 'MAP@1': MAP(k=1, divide_by_k=False),
 'MAP@5': MAP(k=5, divide_by_k=False),
 'MAP@10': MAP(k=10, divide_by_k=False),
 'NDCG@1': NDCG(k=1, log_base=2),
 'NDCG@5': NDCG(k=5, log_base=2),
 'NDCG@10': NDCG(k=10, log_base=2),
 'novelty@1': MeanInvUserFreq(k=1),
 'novelty@5': MeanInvUserFreq(k=5),
 'novelty@10': MeanInvUserFreq(k=10),
 'serendipity@1': Serendipity(k=1),
 'serendipity@5': Serendipity(k=5),
 'serendipity@10': Serendipity(k=10)}

## Splitter

In [22]:
TEST_SIZE = "7D"
N_SPLITS = 5

In [23]:
splitter = TimeRangeSplitter(
    test_size=TEST_SIZE,
    n_splits=N_SPLITS,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

In [24]:
splitter.get_test_fold_borders(dataset.interactions)

[(Timestamp('2021-07-11 00:00:00', freq='7D'),
  Timestamp('2021-07-18 00:00:00', freq='7D')),
 (Timestamp('2021-07-18 00:00:00', freq='7D'),
  Timestamp('2021-07-25 00:00:00', freq='7D')),
 (Timestamp('2021-07-25 00:00:00', freq='7D'),
  Timestamp('2021-08-01 00:00:00', freq='7D')),
 (Timestamp('2021-08-01 00:00:00', freq='7D'),
  Timestamp('2021-08-08 00:00:00', freq='7D')),
 (Timestamp('2021-08-08 00:00:00', freq='7D'),
  Timestamp('2021-08-15 00:00:00', freq='7D'))]

## Cross-val

In [26]:
results = cross_validate(dataset, splitter, metrics, models, k=10, filter_viewed=True)

In [27]:
df_quality = (
    pd.DataFrame.from_dict(results["metrics"]).groupby("model").mean().drop("i_split", axis=1).T
)
df_quality.style.highlight_max(color="lightgreen", axis=1)

model,ALS,LightFM,popular
precision@1,0.093796,0.070746,0.081874
recall@1,0.057927,0.044801,0.050963
precision@5,0.053698,0.047117,0.056047
recall@5,0.154570,0.137203,0.161825
precision@10,0.034668,0.031573,0.036284
recall@10,0.193196,0.178163,0.203790
NDCG@1,0.093796,0.070746,0.081874
NDCG@5,0.062141,0.051808,0.061691
NDCG@10,0.045910,0.039293,0.045989
MAP@1,0.057927,0.044801,0.050963


По большиству метрик **лучшей оказалась ALS**.

# Preparing model for the service

## Training on the whole dataset

Лучшей моделью оказалась ALS. Но, насколько я понимаю, для ее использования нужен GPU.   
Т.к. у меня нет возможности использовать GPU в сервисе, для него я решила взять LightFM, которая не намного уступает ALS.  
Обучим ее на всем датасете.

In [28]:
user_features = get_user_features(users, interactions, ["sex", "age", "income"])
item_features = get_item_features(items, interactions)

In [29]:
dataset = Dataset.construct(
    interactions_df=interactions,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

In [30]:
model = LightFMWrapperModel(
    LightFM(
        no_components=8,
        loss="warp",
        random_state=RANDOM_STATE,
        learning_rate=0.05,
        user_alpha=0.3,
        item_alpha=0.2,
    ),
    epochs=N_EPOCHS,
    num_threads=2,
)

In [31]:
model.fit(dataset)

# Offline recommendations

В чистом виде LightFM не прошла валидацию по времени через бота :(  
Поэтому посчитаем оффлайн рекомендации и сохраним их на диск, чтобы потом использовать в сервисе.

In [36]:
ALL_USERS = interactions[Columns.User].unique()

In [37]:
all_recos = model.recommend(
    users=ALL_USERS,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)[[Columns.User, Columns.Item]]

In [38]:
RECOS_PATH = "/content/drive/MyDrive/recsys/LightFM_warp_8.csv"
all_recos.to_csv(RECOS_PATH)

# Adding ANN

Чтобы рекомендации выдавались быстрее, можно использовать приближенный поиск соседей из `rectools`.

In [32]:
user_vectors, item_vectors = model.get_vectors(dataset)
ann_lightfm = UserToItemAnnRecommender(
    user_vectors=user_vectors,
    item_vectors=item_vectors,
    user_id_map=dataset.user_id_map,
    item_id_map=dataset.item_id_map,
)
ann_lightfm.fit()

In [33]:
# Поробуем получить рекомендации
ann_lightfm.get_item_list_for_user(962205, top_n=10).tolist()

[12138, 10440, 4240, 1679, 6964, 12089, 2150, 12365, 14701, 10843]

In [34]:
import pickle

MODEL_PATH = "/content/drive/MyDrive/recsys/ANN_LightFM_warp_8.pkl"
pickle.dump(ann_lightfm, open(MODEL_PATH, "wb"))

In [35]:
# Проверим, что все работает
loaded_ann_lightfm = pickle.load(open(MODEL_PATH, "rb"))
ann_lightfm.get_item_list_for_user(962205, top_n=10).tolist()

[12138, 10440, 4240, 1679, 6964, 12089, 2150, 12365, 14701, 10843]